In [ ]:
import pandas as pd
import holoviews as hv
import hvplot.pandas
import matplotlib.pyplot as plt
import seaborn as sns
import toml
import re
import urllib
from datetime import datetime
import string
import pygsheets
import requests
import time
from tqdm.auto import tqdm
import Bio.Restriction as Restriction
from Bio.Seq import Seq
import benchlingapi

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.api as api
from paulssonlab.api.util import base_url
import paulssonlab.cloning.workflow as workflow
import paulssonlab.cloning.util as cloning_util
import paulssonlab.cloning.sequence as sequence
import paulssonlab.cloning.golden_gate as golden_gate
import paulssonlab.cloning.codon as codon
import paulssonlab.cloning.optimization as optimization
import paulssonlab.api.twist as twist

In [ ]:
hv.extension("bokeh")

# Setup

In [ ]:
config = toml.load("config.toml")

In [ ]:
session = benchlingapi.Session(config["benchling"]["api_key"])

In [ ]:
gc = pygsheets.authorize(service_account_file="credentials.json")

In [ ]:
col = workflow.get_strain_collection_sheets(gc.drive.service, "LIB")
col

In [ ]:
strain_sheet = gc.open_by_key(col["strains"]).worksheet()
plasmid_sheet = gc.open_by_key(col["plasmids"]).worksheet()
part_sheet = gc.open_by_key(col["parts"]).worksheet()
part_type_sheet = gc.open_by_key(col["parts"]).worksheet_by_title("Part types")

In [ ]:
drive_service = plasmid_sheet.client.drive.service
plasmid_folder = col["plasmid_maps"]
plasmid_maps = api.google.list_drive(drive_service, root=plasmid_folder)

# Twist API

Currently, this has to be run via the HMS network (or via the HMS proxy) because Twist restricts the IP address ranges that can access their API staging server.

In [ ]:
# HOW TO SPECIFY CDS START/END/WHETHER TO ADD STOP
# GG FLANKS
# HOW TO CHECK TWIST FLANKS?

# check where start codon is
# convert AA's -> nt's using E. coli codon usage table
# add stop codons
# add GG flanks (for integration into storage vector)
# check scores
# for all bad scores (AND all that contain RE sites), codon optimize
# codon optimize avoiding: BsaI, BsmBI, BbsI, AarI
# order options: 96-well plate, cap at 1000ug per well
# request quote
# shipping details: Johan/Paulsson/2135454219, share with email: paulssonlab@gmail.com, jqshenker+twist@g.harvard.edu
# checkout
# use existing payment method: BPO #70003066270
# confirm

# download plate map
# add plate map info to oLIB0.x spreadsheet (e.g., oLIB0.7.H12)

In [ ]:
TWIST_AUTHENTICATION_HEADERS = {
    "Authorization": "JWT {}".format(config["twist"]["api_token"]),
    "X-End-User-Token": config["twist"]["end_user_token"],
}

In [ ]:
r = requests.get(
    "https://twist-api.twistbioscience-staging.com/v1/prices/",
    headers=AUTHENTICATION_HEADERS,
)
twist_prices = r.json()

In [ ]:
twist_prices

## Codon optimization

In [ ]:
import pickle

with open("201013voigtsigmas.json", "rb") as f:
    data = pickle.load(f)

In [ ]:
seqs = {
    item["name"]: (item["optimized_seq"], item["cds_location"])
    for item in data["sequences_to_order"].values()
    if item["kind"] != "promoter"
}

In [ ]:
scored_constructs = twist.score(config["twist"], {k: v[0] for k, v in seqs.items()})

In [ ]:
scored_constructs["AS22_1147"]["score"]

In [ ]:
scored_constructs["ECF16_3622"]

In [ ]:
optimized_constructs = twist.score_and_optimize(
    config["twist"],
    seqs,
    avoid_enzymes=["BsaI", "BsmBI", "BbsI", "AarI"],
    organism="Escherichia coli",
)

In [ ]:
optimized_constructs["AS22_1147"]  # ["results"]  # .keys()#["error"]

## Past orders

In [ ]:
r = requests.get(
    "https://twist-api.twistbioscience-staging.com/v1/users/paulssonlab@gmail.com/orders/",
    headers=TWIST_AUTHENTICATION_HEADERS,
)
orders = r.json()

In [ ]:
[(o["order_name"], o["project_name"]) for o in orders]